In [36]:
%load_ext autoreload
%autoreload 2

from notebook import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# # Copie les notebooks et supprime les sorties.
# copy_and_clean_notebooks()

# Chargement des issues à analyser

In [4]:
gh_env = GithubAPIEnv("../P10_04_function/create_github_issue/.env")

In [6]:
# On crée le handler Github
g = Github(gh_env.GITHUB_TOKEN)

# On configure le handler pour le repo du projet
repo = g.get_repo(gh_env.GITHUB_REPO)

In [44]:
# On récupère les issues de type "azure alert"
issues = repo.get_issues(labels=["azure alert"])
issues.totalCount

1

In [45]:
issue_ids = [f"#{i.number}" for i in issues]

print("Voici les ids des issues que l'on va analyser :")
print(", ".join(issue_ids))

Voici les ids des issues que l'on va analyser :
#37


# Chargement des dialogues

In [46]:
ai_env = AppInsightsAPIEnv(".env")

In [47]:
data = []
for issue in issues:
    # On charge les données de l'alerte
    issue_body = json.loads(issue.body)
    
    # On charge les dialogues liés à l'alerte
    data.append(get_tn_dialogs(
        ai_env,
        issue_body["start_dt"],
        issue_body["end_dt"]
    ))

In [48]:
# On concatène les données
data = pd.concat(data)
data.shape

(34, 5)

In [49]:
# On supprime les doublons et on met en forme le dataframe
data = data.drop_duplicates(["main_dialog_uuid", "timestamp"])
data = data.sort_values(["main_dialog_uuid", "timestamp"])
data = data.reset_index(drop=True)
data.shape

(34, 5)

In [50]:
data.head()

,main_dialog_uuid,timestamp,author,session_id,text
0,4b690144-6a3f-11ec-9134-4ae2b1217b29,2021-12-31 13:41:02.230192+00:00,p10-chatbot-bot,I1qjj6+1EFOsJHS9K65QNapgBPr978QJHxKE6iUSeXk=,What else can I do for you?
1,4b690144-6a3f-11ec-9134-4ae2b1217b29,2021-12-31 13:41:06.408448+00:00,You,I1qjj6+1EFOsJHS9K65QNapgBPr978QJHxKE6iUSeXk=,I want to go to Paris from London tomorrow and...
2,4b690144-6a3f-11ec-9134-4ae2b1217b29,2021-12-31 13:41:07.028214+00:00,p10-chatbot-bot,I1qjj6+1EFOsJHS9K65QNapgBPr978QJHxKE6iUSeXk=,Please confirm the following information:\n- Y...
3,4b690144-6a3f-11ec-9134-4ae2b1217b29,2021-12-31 13:41:08.387940+00:00,You,I1qjj6+1EFOsJHS9K65QNapgBPr978QJHxKE6iUSeXk=,Non
4,4f4fdde6-6a3f-11ec-9134-4ae2b1217b29,2021-12-31 13:41:08.776486+00:00,p10-chatbot-bot,I1qjj6+1EFOsJHS9K65QNapgBPr978QJHxKE6iUSeXk=,What else can I do for you?


# Analyse des insatisfactions

In [51]:
analysis_name = "2022_01_01_analysis"

In [52]:
analyser = InsatisfactionsAnalyser(data)
analyser.load(PARQUET_PATH, analysis_name)
analyser.display()

Missing file: data/parquet/2022_01_01_analysis_data.parquet


In [53]:
analyser.save(PARQUET_PATH, analysis_name)

# Enregistrement du rapport sur Github

In [54]:
report_title = "Insatisfactions analysis"
report_body = analyser.get_report(issue_ids)

In [55]:
try:
    # On récupère le tag spécial pour le rapport d'analyse
    report_label_name = "insatisfactions analysis"
    report_label = repo.get_label(report_label_name)
except:
    # On crée un tag spécial pour le rapport d'analyse
    report_label = repo.create_label(
        report_label_name,
        color="0075ca",
        description="Analysis of insatisfactions"
    )

# On enregistre le rapport dans une Github issue
report = repo.create_issue(
    title=report_title,
    body=report_body,
    labels=[report_label]
)

# Fermeture des issues analysées

In [56]:
for issue in issues:
    # On ferme l'issue
    issue.edit(state="closed")